In [1]:
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Activation

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

%matplotlib inline
# plt.style.use("ggplot")
sns.set_style("darkgrid")

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
data = pd.read_csv("final_with_weather.csv")
data.drop(['Month'], axis =1, inplace= True)
data.set_index("pickuptime", inplace= True)
num_pickups = data.total
num_green_pickups = data.green
num_yellow_pickups = data.yellow
data.drop(['green', 'yellow', 'total'], axis =1, inplace = True)

In [3]:
def series_to_supervised(data, n_in, n_out, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
        
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    
    agg = pd.concat(cols, axis = 1)
    agg.columns = names
    
    if dropnan:
        agg.dropna(inplace = True)
        
    return agg

In [4]:
features_and_target = pd.concat([data, num_pickups], axis = 1)
values = features_and_target.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range = (0, 1))
scaled = scaler.fit_transform(values)

n_hours = 23

reframed = series_to_supervised(scaled, n_hours, 1)

n_features = features_and_target.shape[1]


In [5]:
# delete significantly wrong data & compare the date value
total_values = reframed.values
n_p = np.argmin(total_values[365*24*3:, -1])
n_p = n_p + 365*24*3

a = total_values[: n_p, :]
b = total_values[n_p + n_hours + 1 : , :]
total_values = np.concatenate((a, b), axis=0)

# result variable
result = np.zeros((4, 24))

In [ ]:
for i in range (24):
    values = total_values[total_values[:, -9] == total_values[i, 4]]
    n_train_hours = 365*3
    train = values[:n_train_hours, :]
    test = values[n_train_hours:, :]
    # split into input and outputs
    n_obs = n_hours * n_features
    train_X, train_y = train[:, :n_obs], train[:, -1]
    test_X, test_y = test[:, :n_obs], test[:, -1]
    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
    test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

    # design network
    model = Sequential()
    model.add(LSTM(24, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')

    # fit network
    history = model.fit(train_X, train_y, epochs=200, batch_size=train_X.shape[0],
                    validation_data=(test_X, test_y), verbose=2, shuffle=False)
    
    history_loss = pd.DataFrame()
    history_loss['train'] = history.history['loss']
    history_loss['test'] = history.history['val_loss']
    history_loss.plot(figsize=(10,10), fontsize=16,
                title='LSTM Model Loss');
    
    # make a prediction 
    yhat = model.predict(test_X)
    # reshape back to 2d
    test_X = test_X.reshape((test_X.shape[0], -1))

    # invert scaling for forecast# invert 
    inv_yhat = np.concatenate((test[:, 299:311], yhat), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,-1]

    # invert scaling for actual
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = np.concatenate((test[:, 299:311], test_y), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,-1]
    inv_yhat_gte_zero = inv_yhat.copy()
    inv_yhat_gte_zero[inv_yhat_gte_zero < 0] = 0

    # calculate r2
    result[0, i] = r2_score(inv_y, inv_yhat)

    # calculate mean absolute error
    result[1, i] = mean_absolute_error(inv_y, inv_yhat)

    # calculate rmse
    rmse = math.sqrt(mean_squared_error(inv_y, inv_yhat))
    result[2, i]= rmse

    # calculate mean absolute error percentage
    maep = np.mean(np.abs((inv_y - inv_yhat)/inv_y))*100
    result[3, i] = maep

Train on 1095 samples, validate on 274 samples
Epoch 1/200
 - 5s - loss: 0.4150 - val_loss: 0.2829
Epoch 2/200
 - 0s - loss: 0.3620 - val_loss: 0.2319
Epoch 3/200
 - 0s - loss: 0.3093 - val_loss: 0.1822
Epoch 4/200
 - 0s - loss: 0.2570 - val_loss: 0.1402
Epoch 5/200
 - 0s - loss: 0.2067 - val_loss: 0.1131
Epoch 6/200
 - 0s - loss: 0.1635 - val_loss: 0.1032
Epoch 7/200
 - 0s - loss: 0.1342 - val_loss: 0.1074
Epoch 8/200
 - 0s - loss: 0.1218 - val_loss: 0.1223
Epoch 9/200
 - 0s - loss: 0.1216 - val_loss: 0.1421
Epoch 10/200
 - 0s - loss: 0.1284 - val_loss: 0.1613
Epoch 11/200
 - 0s - loss: 0.1374 - val_loss: 0.1765
Epoch 12/200
 - 0s - loss: 0.1439 - val_loss: 0.1844
Epoch 13/200
 - 0s - loss: 0.1467 - val_loss: 0.1851
Epoch 14/200
 - 0s - loss: 0.1450 - val_loss: 0.1792
Epoch 15/200
 - 0s - loss: 0.1395 - val_loss: 0.1681
Epoch 16/200
 - 0s - loss: 0.1312 - val_loss: 0.1531
Epoch 17/200
 - 0s - loss: 0.1212 - val_loss: 0.1359
Epoch 18/200
 - 0s - loss: 0.1111 - val_loss: 0.1183
Epoch 19

Epoch 154/200
 - 0s - loss: 0.0459 - val_loss: 0.0250
Epoch 155/200
 - 0s - loss: 0.0458 - val_loss: 0.0249
Epoch 156/200
 - 0s - loss: 0.0457 - val_loss: 0.0247
Epoch 157/200
 - 0s - loss: 0.0456 - val_loss: 0.0244
Epoch 158/200
 - 0s - loss: 0.0456 - val_loss: 0.0242
Epoch 159/200
 - 0s - loss: 0.0455 - val_loss: 0.0241
Epoch 160/200
 - 0s - loss: 0.0454 - val_loss: 0.0240
Epoch 161/200
 - 0s - loss: 0.0454 - val_loss: 0.0239
Epoch 162/200
 - 0s - loss: 0.0453 - val_loss: 0.0239
Epoch 163/200
 - 0s - loss: 0.0452 - val_loss: 0.0239
Epoch 164/200
 - 0s - loss: 0.0452 - val_loss: 0.0238
Epoch 165/200
 - 0s - loss: 0.0451 - val_loss: 0.0237
Epoch 166/200
 - 0s - loss: 0.0450 - val_loss: 0.0236
Epoch 167/200
 - 0s - loss: 0.0450 - val_loss: 0.0235
Epoch 168/200
 - 0s - loss: 0.0449 - val_loss: 0.0235
Epoch 169/200
 - 0s - loss: 0.0449 - val_loss: 0.0234
Epoch 170/200
 - 0s - loss: 0.0448 - val_loss: 0.0234
Epoch 171/200
 - 0s - loss: 0.0447 - val_loss: 0.0233
Epoch 172/200
 - 0s - loss: 

Epoch 107/200
 - 0s - loss: 0.0586 - val_loss: 0.0394
Epoch 108/200
 - 0s - loss: 0.0586 - val_loss: 0.0397
Epoch 109/200
 - 0s - loss: 0.0585 - val_loss: 0.0399
Epoch 110/200
 - 0s - loss: 0.0584 - val_loss: 0.0401
Epoch 111/200
 - 0s - loss: 0.0584 - val_loss: 0.0402
Epoch 112/200
 - 0s - loss: 0.0583 - val_loss: 0.0402
Epoch 113/200
 - 0s - loss: 0.0583 - val_loss: 0.0400
Epoch 114/200
 - 0s - loss: 0.0582 - val_loss: 0.0398
Epoch 115/200
 - 0s - loss: 0.0582 - val_loss: 0.0396
Epoch 116/200
 - 0s - loss: 0.0581 - val_loss: 0.0394
Epoch 117/200
 - 0s - loss: 0.0581 - val_loss: 0.0393
Epoch 118/200
 - 0s - loss: 0.0580 - val_loss: 0.0393
Epoch 119/200
 - 0s - loss: 0.0580 - val_loss: 0.0393
Epoch 120/200
 - 0s - loss: 0.0579 - val_loss: 0.0394
Epoch 121/200
 - 0s - loss: 0.0579 - val_loss: 0.0396
Epoch 122/200
 - 0s - loss: 0.0578 - val_loss: 0.0397
Epoch 123/200
 - 0s - loss: 0.0578 - val_loss: 0.0397
Epoch 124/200
 - 0s - loss: 0.0577 - val_loss: 0.0397
Epoch 125/200
 - 0s - loss: 

 - 0s - loss: 0.0883 - val_loss: 0.0575
Epoch 60/200
 - 0s - loss: 0.0882 - val_loss: 0.0577
Epoch 61/200
 - 0s - loss: 0.0880 - val_loss: 0.0581
Epoch 62/200
 - 0s - loss: 0.0879 - val_loss: 0.0588
Epoch 63/200
 - 0s - loss: 0.0877 - val_loss: 0.0596
Epoch 64/200
 - 0s - loss: 0.0876 - val_loss: 0.0604
Epoch 65/200
 - 0s - loss: 0.0875 - val_loss: 0.0609
Epoch 66/200
 - 0s - loss: 0.0874 - val_loss: 0.0612
Epoch 67/200
 - 0s - loss: 0.0872 - val_loss: 0.0611
Epoch 68/200
 - 0s - loss: 0.0871 - val_loss: 0.0607
Epoch 69/200
 - 0s - loss: 0.0870 - val_loss: 0.0601
Epoch 70/200
 - 0s - loss: 0.0868 - val_loss: 0.0594
Epoch 71/200
 - 0s - loss: 0.0867 - val_loss: 0.0586
Epoch 72/200
 - 0s - loss: 0.0865 - val_loss: 0.0581
Epoch 73/200
 - 0s - loss: 0.0864 - val_loss: 0.0577
Epoch 74/200
 - 0s - loss: 0.0863 - val_loss: 0.0575
Epoch 75/200
 - 0s - loss: 0.0861 - val_loss: 0.0576
Epoch 76/200
 - 0s - loss: 0.0860 - val_loss: 0.0579
Epoch 77/200
 - 0s - loss: 0.0859 - val_loss: 0.0583
Epoch 

 - 0s - loss: 0.2198 - val_loss: 0.1719
Epoch 12/200
 - 0s - loss: 0.2008 - val_loss: 0.1475
Epoch 13/200
 - 0s - loss: 0.1852 - val_loss: 0.1267
Epoch 14/200
 - 0s - loss: 0.1727 - val_loss: 0.1117
Epoch 15/200
 - 0s - loss: 0.1641 - val_loss: 0.1031
Epoch 16/200
 - 0s - loss: 0.1596 - val_loss: 0.1022
Epoch 17/200
 - 0s - loss: 0.1582 - val_loss: 0.1049
Epoch 18/200
 - 0s - loss: 0.1583 - val_loss: 0.1091
Epoch 19/200
 - 0s - loss: 0.1590 - val_loss: 0.1134
Epoch 20/200
 - 0s - loss: 0.1594 - val_loss: 0.1165
Epoch 21/200
 - 0s - loss: 0.1589 - val_loss: 0.1168
Epoch 22/200
 - 0s - loss: 0.1567 - val_loss: 0.1143
Epoch 23/200
 - 0s - loss: 0.1530 - val_loss: 0.1092
Epoch 24/200
 - 0s - loss: 0.1480 - val_loss: 0.1022
Epoch 25/200
 - 0s - loss: 0.1424 - val_loss: 0.0939
Epoch 26/200
 - 0s - loss: 0.1368 - val_loss: 0.0853
Epoch 27/200
 - 0s - loss: 0.1317 - val_loss: 0.0768
Epoch 28/200
 - 0s - loss: 0.1275 - val_loss: 0.0693
Epoch 29/200
 - 0s - loss: 0.1243 - val_loss: 0.0631
Epoch 

...

Epoch 21/200
 - 0s - loss: 0.0868 - val_loss: 0.1505
Epoch 22/200
 - 0s - loss: 0.0857 - val_loss: 0.1480
Epoch 23/200
 - 0s - loss: 0.0843 - val_loss: 0.1441
Epoch 24/200
 - 0s - loss: 0.0829 - val_loss: 0.1394
Epoch 25/200
 - 0s - loss: 0.0817 - val_loss: 0.1346
Epoch 26/200
 - 0s - loss: 0.0807 - val_loss: 0.1303
Epoch 27/200
 - 0s - loss: 0.0798 - val_loss: 0.1270
Epoch 28/200
 - 0s - loss: 0.0789 - val_loss: 0.1246
Epoch 29/200
 - 0s - loss: 0.0778 - val_loss: 0.1232
Epoch 30/200
 - 0s - loss: 0.0766 - val_loss: 0.1227
Epoch 31/200
 - 0s - loss: 0.0753 - val_loss: 0.1226
Epoch 32/200
 - 0s - loss: 0.0742 - val_loss: 0.1225
Epoch 33/200
 - 0s - loss: 0.0730 - val_loss: 0.1221
Epoch 34/200
 - 0s - loss: 0.0720 - val_loss: 0.1208
Epoch 35/200
 - 0s - loss: 0.0710 - val_loss: 0.1184
Epoch 36/200
 - 0s - loss: 0.0700 - val_loss: 0.1148
Epoch 37/200
 - 0s - loss: 0.0690 - val_loss: 0.1108
Epoch 38/200
 - 0s - loss: 0.0680 - val_loss: 0.1068
Epoch 39/200
 - 0s - loss: 0.0672 - val_loss: 

 - 0s - loss: 0.0425 - val_loss: 0.0766
Epoch 175/200
 - 0s - loss: 0.0424 - val_loss: 0.0772
Epoch 176/200
 - 0s - loss: 0.0423 - val_loss: 0.0768
Epoch 177/200
 - 0s - loss: 0.0422 - val_loss: 0.0770
Epoch 178/200
 - 0s - loss: 0.0420 - val_loss: 0.0767
Epoch 179/200
 - 0s - loss: 0.0419 - val_loss: 0.0763
Epoch 180/200
 - 0s - loss: 0.0418 - val_loss: 0.0766
Epoch 181/200
 - 0s - loss: 0.0417 - val_loss: 0.0761
Epoch 182/200
 - 0s - loss: 0.0417 - val_loss: 0.0769
Epoch 183/200
 - 0s - loss: 0.0417 - val_loss: 0.0761
Epoch 184/200
 - 0s - loss: 0.0415 - val_loss: 0.0766
Epoch 185/200
 - 0s - loss: 0.0413 - val_loss: 0.0762
Epoch 186/200
 - 0s - loss: 0.0411 - val_loss: 0.0759
Epoch 187/200
 - 0s - loss: 0.0410 - val_loss: 0.0764
Epoch 188/200
 - 0s - loss: 0.0411 - val_loss: 0.0758
Epoch 189/200
 - 0s - loss: 0.0411 - val_loss: 0.0765
Epoch 190/200
 - 0s - loss: 0.0407 - val_loss: 0.0765
Epoch 191/200
 - 0s - loss: 0.0406 - val_loss: 0.0755
Epoch 192/200
 - 0s - loss: 0.0407 - val_l

 - 0s - loss: 0.0443 - val_loss: 0.0538
Epoch 128/200
 - 0s - loss: 0.0439 - val_loss: 0.0537
Epoch 129/200
 - 0s - loss: 0.0435 - val_loss: 0.0536
Epoch 130/200
 - 0s - loss: 0.0432 - val_loss: 0.0533
Epoch 131/200
 - 0s - loss: 0.0428 - val_loss: 0.0529
Epoch 132/200
 - 0s - loss: 0.0425 - val_loss: 0.0524
Epoch 133/200
 - 0s - loss: 0.0421 - val_loss: 0.0520
Epoch 134/200
 - 0s - loss: 0.0418 - val_loss: 0.0520
Epoch 135/200
 - 0s - loss: 0.0414 - val_loss: 0.0520
Epoch 136/200
 - 0s - loss: 0.0411 - val_loss: 0.0518
Epoch 137/200
 - 0s - loss: 0.0408 - val_loss: 0.0513
Epoch 138/200
 - 0s - loss: 0.0405 - val_loss: 0.0509
Epoch 139/200
 - 0s - loss: 0.0402 - val_loss: 0.0508
Epoch 140/200
 - 0s - loss: 0.0399 - val_loss: 0.0509
Epoch 141/200
 - 0s - loss: 0.0397 - val_loss: 0.0509
Epoch 142/200
 - 0s - loss: 0.0394 - val_loss: 0.0508
Epoch 143/200
 - 0s - loss: 0.0392 - val_loss: 0.0506
Epoch 144/200
 - 0s - loss: 0.0390 - val_loss: 0.0506
Epoch 145/200
 - 0s - loss: 0.0388 - val_l

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Train on 1095 samples, validate on 272 samples
Epoch 1/200
 - 3s - loss: 0.3237 - val_loss: 0.1866
Epoch 2/200
 - 0s - loss: 0.2811 - val_loss: 0.1460
Epoch 3/200
 - 0s - loss: 0.2384 - val_loss: 0.1075
Epoch 4/200
 - 0s - loss: 0.1957 - val_loss: 0.0750
Epoch 5/200
 - 0s - loss: 0.1542 - val_loss: 0.0549
Epoch 6/200
 - 0s - loss: 0.1174 - val_loss: 0.0544
Epoch 7/200
 - 0s - loss: 0.0913 - val_loss: 0.0745
Epoch 8/200
 - 0s - loss: 0.0785 - val_loss: 0.1034
Epoch 9/200
 - 0s - loss: 0.0802 - val_loss: 0.1317
Epoch 10/200
 - 0s - loss: 0.0900 - val_loss: 0.1532
Epoch 11/200
 - 0s - loss: 0.1012 - val_loss: 0.1662
Epoch 12/200
 - 0s - loss: 0.1092 - val_loss: 0.1713
Epoch 13/200
 - 0s - loss: 0.1124 - val_loss: 0.1695
Epoch 14/200
 - 0s - loss: 0.1111 - val_loss: 0.1624
Epoch 15/200
 - 0s - loss: 0.1062 - val_loss: 0.1513
Epoch 16/200
 - 0s - loss: 0.0992 - val_loss: 0.1373
Epoch 17/200
 - 0s - loss: 0.0913 - val_loss: 0.1217
Epoch 18/200
 - 0s - loss: 0.0836 - val_loss: 0.1059
Epoch 19

In [ ]:
result[3, :]